In [1]:
import pandas as pd
import numpy as np

# Indexing

## loc: label-based indexing

In [2]:
np.random.seed(0)
df1 = pd.DataFrame(np.random.randn(6,4), index=list('abcdef'), columns=list('ABCD'))
df1

,A,B,C,D
a,1.764052,0.400157,0.978738,2.240893
b,1.867558,-0.977278,0.950088,-0.151357
c,-0.103219,0.410599,0.144044,1.454274
d,0.761038,0.121675,0.443863,0.333674
e,1.494079,-0.205158,0.313068,-0.854096
f,-2.552990,0.653619,0.864436,-0.742165


In [3]:
df1.loc[['a', 'c'], ['A', 'C']]

,A,C
a,1.764052,0.978738
c,-0.103219,0.144044


In [4]:
# returns Series
df1.loc['a', ['A', 'C']]

A    1.764052
C    0.978738
Name: a, dtype: float64

## iloc: integer positional indexing

In [5]:
# Get 1st, 2nd, and 4th row and 1st and 2nd column: 
df1.iloc[[0, 1, 3], [0, 1]]

# Get all rows and columns 2-9 if present (range is exclusive)
df1.iloc[:, 2:10]

# Get 2nd and 3rd row
df1.iloc[1:3]

,A,B,C,D
b,1.867558,-0.977278,0.950088,-0.151357
c,-0.103219,0.410599,0.144044,1.454274


## Boolean mask indexing

In [6]:
df1[(df1.A < 0) & (df1['C'] > 0)]

,A,B,C,D
c,-0.103219,0.410599,0.144044,1.454274
f,-2.552990,0.653619,0.864436,-0.742165


In [7]:
# SQL: select C from df where A < 1
df1.loc[df1.A < 1, 'C']

c    0.144044
d    0.443863
f    0.864436
Name: C, dtype: float64

# Grouping

In [8]:
np.random.seed(0)
df2 = pd.DataFrame({'A' : ['foo', 'foo', 'foo', 'bar', 'bar'],
                    'B' : ['one', 'one', 'two', 'two', 'two'],
                    'C' : [1, 2, 3, 4, 5],
                    'D' : [1, 1, 1, 1, 1]}) 
df2

,A,B,C,D
0,foo,one,1,1
1,foo,one,2,1
2,foo,two,3,1
3,bar,two,4,1
4,bar,two,5,1


In [9]:
# Series with MultiIndex (A,B) and values=max(C) per A
s = df2.groupby(['A','B'])['C'].sum()
s.to_frame('sum_C')

sum_C
A   B         
bar two      9
foo one      3
    two      3

In [10]:
df2.groupby(['A','B']).agg({'C': sum, 'D': lambda x: len(x)})

C  D
A   B        
bar two  9  2
foo one  3  2
    two  3  1

In [11]:
# Series with index=order_id (like index before) and values=max(order_number)
# Data has the same number as rows as before, i.e. groupby('user_id') does not return one row per user_id!
df2['E'] = df2.groupby('A')['C'].transform(np.mean)
df2

,A,B,C,D,E
0,foo,one,1,1,2.0
1,foo,one,2,1,2.0
2,foo,two,3,1,2.0
3,bar,two,4,1,4.5
4,bar,two,5,1,4.5
